In [1]:
from utilities.utils_clean import *
from utilities.test_dataset_utils import *

In [ ]:

#download_patents_pto(start_year=2015, end_year=2015, kind='grant',download_path="data")
#unzip_files("data","patent_grants_2015")

In [ ]:
# # Example usage
# test_dataset = create_test_dataset_from_freilich(
#     year=2015,  # The year you want to analyze
#     freilich_data_path="../data/Freilich.Data.Compressed.xlsb",  # Path to your Freilich dataset
#     path_to_all_xmls_for_chosen_year="../data/patent_grants_2015"  # Directory containing XML files
# )

# # Optional: Check the results
# print(f"Number of patents extracted: {len(test_dataset)}")
# print("Sample document numbers:", list(test_dataset.keys())[:5])

Processing ipg150106.xml
Processing ipg150113.xml
Processing ipg150120.xml
Processing ipg150127.xml
Processing ipg150203.xml
Processing ipg150210.xml
Processing ipg150217.xml
Processing ipg150224.xml
Processing ipg150303.xml
Processing ipg150310.xml
Processing ipg150317.xml
Processing ipg150324.xml
Processing ipg150331.xml
Processing ipg150407.xml
Processing ipg150414.xml
Processing ipg150421.xml
Processing ipg150428.xml
Processing ipg150505.xml
Processing ipg150512.xml
Processing ipg150519.xml
Processing ipg150526.xml
Processing ipg150602.xml
Processing ipg150609.xml
Processing ipg150616.xml
Processing ipg150623.xml
Processing ipg150630.xml
Processing ipg150707.xml
Processing ipg150714.xml
Processing ipg150721.xml
Processing ipg150728.xml
Processing ipg150804.xml
Processing ipg150811.xml
Processing ipg150818.xml
Processing ipg150825.xml
Processing ipg150901.xml
Processing ipg150908.xml
Processing ipg150915.xml
Processing ipg150922.xml
Processing ipg150929.xml
Processing ipg151006.xml


In [2]:
test_data = load_from_pickle("../data/test_dataset_2015.pkl")

df_test_data = pd.DataFrame(test_data,index=["xml"],).T.reset_index()
df_test_data.columns = ["patentnumber","xml"] 

Loaded 25081 patents from ../data/test_dataset_2015.pkl


In [3]:
# df = read_xlsb_file()
# df[(df.issueyear == 2015) | (df.prophetic !=0)][["patentnumber","prophetic","nonprophetic","issueyear"]]
# df[df.issueyear == 2015].to_csv("freilichdataet_2015.csv")
df = pd.read_csv("../data/freilichdataet_2015.csv")
df["patentnumber"] = df["patentnumber"].astype(str).transform(lambda x: x.replace(".0", ""))
df["patentnumber"] = df["patentnumber"].apply(remove_leadiong_zeros)
df.head()

,Unnamed: 0,patentnumber,appyear,issueyear,inventorcountry,divisional,continuation,prioritydate,priority.formatted,priorityyear,...,yr4,yr8,yr11,claims,wordaverage,litigated,orangebook,forward.cites,industry,subindustry
0,497653,8969641,1976.0,2015.0,US,0.0,0.0,19760903.0,28006.0,1976.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,19.0
1,497654,9134302,1995.0,2015.0,US,1.0,1.0,19830505.0,30441.0,1983.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN
2,497655,8926966,1995.0,2015.0,US,1.0,1.0,19900921.0,33137.0,1990.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,3.0,31.0
3,497656,9181140,1993.0,2015.0,US,0.0,1.0,19920916.0,33863.0,1992.0,...,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN
4,497657,9199886,2009.0,2015.0,US,0.0,1.0,19940119.0,34353.0,1994.0,...,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN


In [4]:
merged = df_test_data.merge(df, on="patentnumber", how="left")
print(len(merged))
cols = ["patentnumber","xml","prophetic","nonprophetic","allprophetic","someprophetic"]
merged[cols].head()

25081


,patentnumber,xml,prophetic,nonprophetic,allprophetic,someprophetic
0,RE045323,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...",1.0,12.0,0.0,1.0
1,RE045324,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...",0.0,8.0,0.0,0.0
2,RE045325,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...",0.0,8.0,0.0,0.0
3,8925349,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...",26.0,0.0,1.0,0.0
4,8925551,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...",6.0,5.0,0.0,1.0


In [119]:

doc_w_exp = {}
i = 0
for i,xml in enumerate(merged["xml"]):
    i += 1
    if i % 1000 == 0:
        print(f"{i}/{len(merged.xml)} so far found {len(doc_w_exp)} docs with experiments")
    heading = extract_experiments_w_heading(xml)
    if heading:
        extracted_examples = []
        if len(heading) >1:
            print(f"has more than 1 Examples heading , {i}")
            # for example in heading:
            #     extracted_ex_start_w_word = process_siblings(example.find_next_siblings())
            #     if extracted_ex_start_w_word and len(extracted_ex_start_w_word) > 0:
            #         extracted_examples.append(extracted_ex_start_w_word)
            #     else:
            #         extracted_ex_w_word = extract_examples_start_w_word(example.find_next_siblings())
            #         if extracted_ex_w_word:
            #             if len(extracted_ex_w_word) > 0:
            #                 extracted_examples.append(extracted_ex_w_word)
            #         else:
            #             num_dot_examples = extract_num_dot_examples(str(heading[0].find_next_siblings()))
            #             if len(num_dot_examples) > 0:
            #                 extracted_examples.append(num_dot_examples)
        elif len(heading) == 1:
            example_start_w = process_siblings(heading[0].find_next_siblings())
            if example_start_w:
                if len(example_start_w[0]["content"]) == 0:
                    extracted_ex_w_word = extract_examples_start_w_word(heading[0].find_next_siblings())
                    try:
                        if len(extracted_ex_w_word["content"]) > 0:
                            extracted_examples.append(extracted_ex_w_word)
                    except:
                        if len(extracted_ex_w_word[0]["content"]) > 0:
                            extracted_examples.append(extracted_ex_w_word)
                else:
                    extracted_examples.append(example_start_w)
            else:
                extracted_ex_w_word = extract_examples_start_w_word(heading[0].find_next_siblings())
                if extracted_ex_w_word:
                    if len(extracted_ex_w_word["content"]) > 0:
                        extracted_examples.append(extracted_ex_w_word)
                else:
                    num_dot_examples = extract_num_dot_examples(str(heading[0].find_next_siblings()))
                    if len(num_dot_examples) > 0:
                        extracted_examples.append(num_dot_examples)
        else:
            extracted_ex_w_word = extract_examples_w_word(xml)
            if extracted_ex_w_word:
                if len(extracted_ex_w_word["content"]) > 0:
                    extracted_examples.append(extracted_ex_w_word)
                else:
                    example_start_w = process_siblings(heading[0].find_next_siblings())
                    if example_start_w:
                        if len(example_start_w[0]["content"]) > 0:
                            extracted_examples.append(example_start_w)

            # else:
            #     num_dot_examples = extract_num_dot_examples(str(heading[0].find_next_siblings()))
            #     extracted_examples.append(num_dot_examples)
    
        
    
        if len(extracted_examples) > 0:
            doc_w_exp[find_doc_number(xml)[0]] = extracted_examples
            # if len(doc_w_exp) == 100:
            #     break

has more than 1 Examples heading , 349
has more than 1 Examples heading , 388
has more than 1 Examples heading , 689
has more than 1 Examples heading , 768
has more than 1 Examples heading , 781
has more than 1 Examples heading , 814
1000/25081 so far found 507 docs with experiments
has more than 1 Examples heading , 1054
has more than 1 Examples heading , 1417
has more than 1 Examples heading , 1475


TypeError: list indices must be integers or slices, not str

In [84]:
def extract_examples_from_heading(heading):
    extracted_examples = []
    example_start_w = process_siblings(heading.find_next_siblings())
    if example_start_w:
        if not example_start_w[0]["content"]:
            extracted_ex_w_word = extract_examples_start_w_word(heading.find_next_siblings())
            if extracted_ex_w_word:
                if isinstance(extracted_ex_w_word, list):
                    if extracted_ex_w_word and len(extracted_ex_w_word[0]["content"])>0:
                        extracted_examples.append(extracted_ex_w_word)
                elif extracted_ex_w_word["content"]:
                    extracted_examples.append(extracted_ex_w_word)
        else:
            if len(example_start_w[0]["content"])>0:
                extracted_examples.append(example_start_w)
    else:
        extracted_ex_w_word = extract_examples_start_w_word(heading.find_next_siblings())
        if extracted_ex_w_word:
            if isinstance(extracted_ex_w_word, list):
                if extracted_ex_w_word and len(extracted_ex_w_word[0]["content"])>0:
                    extracted_examples.append(extracted_ex_w_word)
            elif extracted_ex_w_word["content"]:
                if len(extracted_ex_w_word["content"])>0:
                    extracted_examples.append(extracted_ex_w_word)
        else:
            num_dot_examples = extract_num_dot_examples(str(heading.find_next_siblings()))
            if num_dot_examples:
                extracted_examples.append(num_dot_examples)
    return extracted_examples

def extract_examples(xml):
    heading = extract_experiments_w_heading(xml)
    if heading:
        if len(heading) > 1:
            print(f"has more than 1 Examples heading, {i}")
        elif len(heading) == 1:
            return extract_examples_from_heading(heading[0])
        else:
            extracted_ex_w_word = extract_examples_w_word(xml)
            if extracted_ex_w_word:
                if isinstance(extracted_ex_w_word, list):
                    if extracted_ex_w_word and extracted_ex_w_word[0]["content"]:
                        return [extracted_ex_w_word]
                elif extracted_ex_w_word["content"]:
                    return [extracted_ex_w_word]
            else:
                example_start_w = process_siblings(heading[0].find_next_siblings())
                if example_start_w and example_start_w[0]["content"]:
                    return [example_start_w]
    else: 
        example_start_w = extract_examples_w_word(xml)
        if example_start_w:
            return [example_start_w]
        else:
            num_dot_examples = extract_num_dot_examples(xml)
            if num_dot_examples:
                return [num_dot_examples]
    return []

doc_w_exp = {}
for i, xml in enumerate(merged["xml"], start=1):
    if i % 1000 == 0:
        print(f"{i}/{len(merged.xml)} so far found {len(doc_w_exp)} docs with experiments")
    
    extracted_examples = extract_examples(xml)
    if extracted_examples:
        doc_w_exp[find_doc_number(xml)[0]] = extracted_examples

KeyboardInterrupt: 

In [93]:

doc_w_exp = {}
for i, xml in enumerate(merged[:1999].xml.values, start=1):
    if i % 1000 == 0:
        print(f"{i}/{len(merged.xml)} so far found {len(doc_w_exp)} docs with experiments")
    
    extracted_examples = extract_examples(xml)
    if extracted_examples:
        doc_w_exp[find_doc_number(xml)[0]] = extracted_examples

has more than 1 Examples heading, 349
has more than 1 Examples heading, 388
has more than 1 Examples heading, 689
has more than 1 Examples heading, 768
has more than 1 Examples heading, 781
has more than 1 Examples heading, 814
1000/25081 so far found 769 docs with experiments
has more than 1 Examples heading, 1054
has more than 1 Examples heading, 1417
has more than 1 Examples heading, 1475
has more than 1 Examples heading, 1810


In [5]:
# save_as_pickle(doc_w_exp,"doc_w_exp_new_algorithm.pkl")
doc_w_exp = load_from_pickle("doc_w_exp_new_algorithm.pkl")

# save_as_json(doc_w_exp, "doc_w_exp_2015.json")
# doc_w_exp = read_json("../data/doc_w_exp_2015.json")

Loaded 18977 patents from doc_w_exp_new_algorithm.pkl


In [94]:
import re
import html
import unicodedata
from bs4 import BeautifulSoup

def clean_text_updated(text):
    """
    Clean text by removing HTML tags, special characters, extra spaces,
    and normalizing the content while keeping meaningful punctuation.

    Args:
        text (str): Input text to clean

    Returns:
        str: Cleaned text
    """
    if not isinstance(text, str):
        return ""

    # Decode HTML entities & remove HTML tags
    text = html.unescape(text)
    text = BeautifulSoup(text, "html.parser").get_text()

    # Normalize Unicode characters (e.g., é → e)
    text = unicodedata.normalize("NFKC", text)

    # Remove unwanted special characters but keep punctuation
    text = re.sub(r"[^\w\s.,!?'\-]", "", text)

    # Normalize spaces: remove multiple spaces, newlines, and tabs
    text = re.sub(r"\s+", " ", text).strip()

    return text

import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

def check_tense_nltk_updated(sentence):
    words = word_tokenize(sentence)
    tagged = pos_tag(words)

    # Define tense categories
    past = {"VBD", "VBN"}
    present = {"VB", "VBG", "VBP", "VBZ"}
    future = {"MD"}

    tenses = {"past": 0, "present": 0, "future": 0}

    for i, (word, tag) in enumerate(tagged):
        # Count past tense words
        if tag in past:
            tenses["past"] += 1

        # Count present tense words
        elif tag in present:
            tenses["present"] += 1

        # Future tense handling
        elif tag in future:
            if word.lower() in {"will", "shall"}:
                # Ensure 'will' or 'shall' is followed by a verb (to confirm future tense)
                if i + 1 < len(tagged) and tagged[i + 1][1] in {"VB", "VBP"}:
                    tenses["future"] += 1

    return max(tenses, key=tenses.get) if max(tenses.values()) > 0 else "Unknown"



In [95]:
import re
from utilities.nlp_processing import check_tense_nltk,clean_text

def tense_with_pattern(description):
    future_pattern = r'\b(?:will|would|shall|should)\b'
    past_pattern = r'\b(?:was|were|had|did)\b'

    future_matches = re.findall(future_pattern, description)
    past_matches = re.findall(past_pattern, description)
    if future_matches and past_matches:
        if len(future_matches) > len(past_matches):
            return "present"
        else:
            return "past"
    if future_pattern or past_pattern:
        if future_pattern:
            return "present"
        else:
            return "past"
    return "Unkown"

def dic_to_dic_w_tense(doc_w_exp):
    dic = {}
    pattern = r'\(\d+\)\s*([A-Za-z0-9\-\(\)\{\},:;=\[\]\+\*\s\.\^\$\%]+(?:\.(?:sup|delta|Hz|NMR)[^\)]*)?)'
    pattern2 = r'\(\d+\)\s*(?:[A-Za-z]+\s*\d*\-?[A-Za-z]*[\(\{][^)]*[\)\}][^;]*|(?:\.\d*H\-?NMR[^;]*|\d+\.[a-zA-Z]*\d*))'


    for key, value in doc_w_exp.items():
        tense_counts = {"past": 0, "present": 0,"Unknown":0}
        
        if isinstance(value[0], list) and len(value[0]) == 1:
            desc = value[0][0]["title"] + "." + "".join(value[0][0]["content"])
            tense = check_tense_nltk_updated(clean_text(desc))
            if tense !=  "Unknown":
                tense_counts[tense] += 1
            else:
                matches = re.findall(pattern, desc)
                if matches:
                    tense_counts["past"] += 1
                else:
                    tense_counts["Unknown"] += 1
            dic[key] = tense_counts


        elif isinstance(value[0], list) and len(value[0]) > 1:
            for ls in value[0]:
                desc = ls["title"] + "." + "".join(ls["content"])
                tense = check_tense_nltk_updated(clean_text(desc))
                if tense !=  "Unknown":
                    tense_counts[tense] += 1
                else:
                    matches = re.findall(pattern, desc)
                    if matches:
                        tense_counts["past"] += 1
                    else:
                        tense_counts["Unknown"] += 1
            dic[key] = tense_counts

        elif isinstance(value[0], dict):
            for ex, description in value[0].items():
                tense = check_tense_nltk_updated(clean_text(description))
                if tense !=  "Unknown":
                    tense_counts[tense] += 1
                else:
                    matches = re.findall(pattern, description)
                    if matches:
                        tense_counts["past"] += 1
                    else:
                        tense_counts["Unknown"] += 1
            dic[key] = tense_counts

    return dic

import re

def dic_to_dic_w_unknown_tense(doc_w_exp):
    dic = {}
    pattern = r'\(\d+\)\s*([A-Za-z0-9\-\(\)\{\},:;=\[\]\+\*\s\.\^\$\%]+(?:\.(?:sup|delta|Hz|NMR)[^\)]*)?)'
    pattern2 = r'\(\d+\)\s*(?:[A-Za-z]+\s*\d*\-?[A-Za-z]*[\(\{][^)]*[\)\}][^;]*|(?:\.\d*H\-?NMR[^;]*|\d+\.[a-zA-Z]*\d*))'

    for key, value in doc_w_exp.items():
        unknown_experiments = []  # Initialize a list to store the experiments classified as Unknown
        
        if isinstance(value[0], list) and len(value[0]) == 1:
            desc = value[0][0]["title"] + "." + "".join(value[0][0]["content"])
            tense = check_tense_nltk_updated(clean_text(desc))
            if tense == "Unknown":
                unknown_experiments.append(desc)

        elif isinstance(value[0], list) and len(value[0]) > 1:
            for ls in value[0]:
                desc = ls["title"] + "." + "".join(ls["content"])
                tense = check_tense_nltk_updated(clean_text(desc))
                if tense == "Unknown":
                    unknown_experiments.append(desc)

        elif isinstance(value[0], dict):
            for ex, description in value[0].items():
                tense = check_tense_nltk_updated(clean_text(description))
                if tense == "Unknown":
                    unknown_experiments.append(description)

        # Only add to the dictionary if there are "Unknown" experiments
        if unknown_experiments:
            dic[key] = unknown_experiments

    return dic


In [105]:
# 8946300 example 1-5
# 08946296 example 2 onwards 
# 08932484 examples 2 to 11


# 08933074:  
# Synthesis of 5,5-dimethyl-4-(pyridin-4-yl)-3-(4-(quinolin-2-ylmethoxy)phenyl) furan-2(5H)-one (Example 23)
# 5,5-dimethyl-4-(pyridin-4-yl)-3-(4-(quinolin-2-ylmethoxy)phenyl) furan-2(5H)-one (Example 23)

# 08945522:
# Example 2
# (69) 3-(3-(Triethoxysilyl)propylamino)phenyl benzoate and
# (140) Example 9
# (141) 4-(3-(Triethoxysilyl)propylamino)phenyl benzoate and
# (142) Example 10
# (143) 4-(Bis(3-(triethoxysilyl)propyl)amino)phenyl benzoate



SyntaxError: invalid syntax (1140748136.py, line 1)

In [96]:
list_w_exp = list(doc_w_exp.items())
final = dic_to_dic_w_unknown_tense(dict(list_w_exp))

c:\ProgramData\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [98]:
final.keys()

dict_keys(['RE045323', '08926732', '08926736', '08926740', '08926872', '08926918', '08926932', '08926944', '08926947', '08926955', '08927001', '08927010', '08927015', '08927023', '08927025', '08927075', '08927096', '08927098', '08927116', '08927121', '08927240', '08927452', '08927457', '08927466', '08927471', '08927484', '08927513', '08927525', '08927526', '08927533', '08927536', '08927546', '08927555', '08927557', '08927565', '08927571', '08927572', '08927583', '08927588', '08927631', '08927674', '08927698', '08927709', '08927710', '08927720', '08927721', '08927727', '08927729', '08927743', '08927746', '08927781', '08929978', 'RE045336', '08931476', '08932383', '08932400', '08932406', '08932480', '08932481', '08932484', '08932485', '08932490', '08932491', '08932492', '08932558', '08932566', '08932570', '08932573', '08932579', '08932582', '08932595', '08932605', '08932610', '08932622', '08932623', '08932625', '08932626', '08932628', '08932637', '08932642', '08932665', '08932685', '0893

In [140]:
final["08945522"]

['3-(3-(Triethoxysilyl)propylamino)phenyl benzoate and.',
 '4-(3-(Triethoxysilyl)propylamino)phenyl benzoate and.',
 '3-(3-Triethoxysilyl)propylcarbamoyloxy)phenyl benzoate particles (P2).',
 '3-(3-Triethoxysilyl)propylamino)phenyl benzoate particles (P3).',
 '3-(Bis(3-(triethoxysilyl)propyl)aminophenyl benzoate particles (P4).',
 '3-(3-(3-Triethoxysilyl)propyl)ureido)phenyl benzoate particles (P5).',
 '4-((3-(Triethoxysilyl)propoxy)methyl)phenyl benzoate particles (P6).',
 '4-((3-(Triethoxysilyl)propylcarbamoyloxy)methyl)phenyl benzoate (P7).',
 '4-(3-(Triethoxysilyl)propoxy)phenyl benzoate particles (P8).',
 '4-(3-(Triethoxysilyl)propylcarbamoyloxy)phenyl benzoate particles (P9).',
 '4-(3-(Triethoxysilyl)propylamino)phenyl benzoate particles (P10).',
 '4-(Bis(3-(Triethoxysilyl)propylamino)phenyl benzoate particles (P11).']

In [83]:
doc_w_exp["08945728"]

[[{'number': 'Synthesis Example for Compound (C-1)',
   'title': 'Synthesis of Intermediate 2',
   'content': ['',
    'A solution of intermediate 1 (21.0 g, 67.9 mmol) in dry tetrahydrofuran (THF) (210 mL) was cooled to −78° C. Thereafter, a hexane solution of n-butyllithium (1.6 M, 42 mL) was added dropwise thereto, and the resultant mixture was stirred for 1 hour to obtain a lithium reagent. To a solution of cyanuric chloride (25.0 g, 136 mmol) in dry THF (250 mL) was dropwise added the lithium reagent prepared previously. This mixture was stirred for 3 hours. The resultant reaction solution was poured into ice water (300 mL), and the pH thereof was regulated to below 4 with 1 N hydrochloric acid. This mixture was extracted twice with methylene chloride (100 mL). The organic layer was washed twice with saturated aqueous sodium chloride solution (100 mL) and dried with magnesium sulfate. Thereafter, the solvent was distilled off under reduced pressure. The residue was purified by sil

In [48]:
list_w_exp = list(doc_w_exp.items())[:2000]
final = dic_to_dic_w_tense(dict(list_w_exp))
df_final = pd.DataFrame(final).T.reset_index()
df_final.columns = ["patentnumber","past","present","Unknown"]
df_final["patentnumber"] = df_final["patentnumber"].apply(remove_leadiong_zeros)

c:\ProgramData\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [9]:
# extract_examples(merged[merged["patentnumber"] == "9018332"]["xml"].values[0])

In [ ]:
[8969641,8926966,8951515,8940436,9023999,9011956,9018396,9213027,9108964,9084992]#examples our algorithm didnt find but freilich did 
# many dont have an overarching examples section but have example 1, example 2 etc
# check these xml, the text has examples section:9194008,9018332,9054322,9150561,9213027, 

In [10]:
# import pprint as p
# p.pprint(merged[merged.patentnumber == "9018332"]["xml"].values[0])

In [11]:
def extract_examples_start_w_word(xml_siblings):
    examples = []
    current_example = None
    in_example = False

    for tag in xml_siblings:
        if tag.name == "heading":
            text_lower = tag.text.strip().lower()
            if (
                "example" in text_lower
                or "experiment" in text_lower
                or "test" in text_lower
            ):
                in_example = True
                current_example = {
                    "number": tag.text.strip(),
                    "title": xml_siblings[xml_siblings.index(tag) + 1].text.strip(),
                    "content": [],
                }
                examples.append(current_example)
            else:
                # If we hit any other heading, stop collecting content
                in_example = False
        elif in_example and tag.name == "p" and current_example is not None:
            current_example["content"].append(tag.text.strip())

    return examples

In [93]:
merged[merged.patentnumber.isin(["9194008","9018332","9054322","9150561","9213027"])]

,patentnumber,xml,Unnamed: 0,appyear,issueyear,inventorcountry,divisional,continuation,prioritydate,priority.formatted,...,yr4,yr8,yr11,claims,wordaverage,litigated,orangebook,forward.cites,industry,subindustry
8012,9018332,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...",503004,2009.0,2015.0,JP,0.0,0.0,20080115.0,39462.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,15.0
10978,9054322,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...",503043,2012.0,2015.0,JP,0.0,1.0,20081125.0,39777.0,...,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN
19010,9150561,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...",522725,2014.0,2015.0,CH,1.0,0.0,20040112.0,37998.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN
22755,9194008,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...",497659,2013.0,2015.0,US,1.0,1.0,19951115.0,35018.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN
24211,9213027,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...",522705,2013.0,2015.0,US,1.0,0.0,20030730.0,37832.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN


In [51]:
import numpy as np

df_check = df_final.merge(merged,on= "patentnumber",how="left")[["patentnumber","present","past","Unknown","prophetic","nonprophetic","allprophetic","someprophetic"]]
#df_check["past"] = df_check["past"] + df_check["Unknown"]
df_check["Total_Extracted"] = df_check["past"] + df_check["present"]  #+ df_check["Unknown"]
df_check["Total_Freilich"] = df_check["prophetic"] + df_check["nonprophetic"]
df_check["prophetic_error"] = np.sqrt((df_check["prophetic"] - df_check["present"])**2)
df_check["nonprophetic_error"] = np.sqrt((df_check["nonprophetic"] - df_check["past"])**2)
df_check["Total_Mean_error"] = np.sqrt((df_check["Total_Freilich"] - df_check["Total_Extracted"])**2)
df_check["Sum_error"] = df_check["prophetic_error"] + df_check["nonprophetic_error"] + df_check["Total_Mean_error"]
print(f"Number of exact matches: {len(df_check[(df_check.prophetic_error ==0) & (df_check.nonprophetic_error ==0) & (df_check.Total_Mean_error ==0)])} out of {len(df_check)} , Percentage: {len(df_check[(df_check.prophetic_error ==0) & (df_check.nonprophetic_error ==0) & (df_check.Total_Mean_error ==0)])/len(df_check)*100}")
print(f"Number of exact num of patent extracted: {len(df_check[(df_check.Total_Extracted == df_check.Total_Freilich)])} out of {len(df_check)}")
print(f"Avg Total error: {df_check['Total_Mean_error'].mean()}, num of corrects:  {df_check[df_check['Total_Mean_error'] == 0].shape[0]}")
print(f"Avg Total prophetic error: {df_check['prophetic_error'].mean()}, num of corrects: {df_check[df_check['prophetic_error'] == 0].shape[0]}")
print(f"Avg Total nonprophetic error: {df_check['nonprophetic_error'].mean()}, num of corrects: {df_check[df_check['nonprophetic_error'] == 0].shape[0]}")
print(f"{df_check['Sum_error'].sum()}")
df_check.sort_values("Total_Mean_error",ascending=False).head(20)



#With clean_text and tense updated description for first 2000
# Number of exact matches: 626 out of 2000 , Percentage: 31.3
# Number of exact num of patent extracted: 1254 out of 2000
# Avg Total error: 2.1615, num of corrects:  1254
# Avg Total prophetic error: 1.4895, num of corrects: 1041
# Avg Total nonprophetic error: 3.5455, num of corrects: 733
# 14393.0


# not extracting reference examples
# Number of exact matches: 641 out of 2000 , Percentage: 32.05
# Number of exact num of patent extracted: 1274 out of 2000
# Avg Total error: 1.717, num of corrects:  1274
# Avg Total prophetic error: 1.483, num of corrects: 1045
# Avg Total nonprophetic error: 3.126, num of corrects: 750
# 12652.0


# not extracting reference and preparation examples
# Number of exact matches: 650 out of 2000 , Percentage: 32.5
# Number of exact num of patent extracted: 1293 out of 2000
# Avg Total error: 1.561,f num of corrects:  1293
# Avg Total prophetic error: 1.4595, num of corrects: 1049
# Avg Total nonprophetic error: 3.028, num of corrects: 751
# 12097.0


# I have enhanced the algorithm with a different combination of the techniques mentioned in previous email, now our algorithm finds examples in 75.6% 
# of janets dataset for 2015 which is a 25% increase compared to previous algorthmic approach.( from the 25081 patents in janets for 2015 we found 18977 patents with examples)
#After removing all the unknowsn: we have 1227 out of 2000 (61%) patents with the number of examples as janet. The total error calculated as before is now at 17778. which is substantially bigger than before
#in 86 cases we have less than freilich, in 687 cases we have more than freilich (e.g refrence,test,production examples)
# we extracted the same number of prophetics as janets in 50% of the patents where we extracted the same number of total patents as freilich. 613/1227 
# ( i rerun the code for 5000 patents and got 79.5% patents with the same number of prophetics as frelich, and got 3058/5000 with the same total number of patents as janets)

# Number of exact matches: 613 out of 2000 , Percentage: 30.65
# Number of exact num of patent extracted: 1227 out of 2000
# Avg Total error: 3.0625, num of corrects:  1227
# Avg Total prophetic error: 1.8045, num of corrects: 961
# Avg Total nonprophetic error: 4.022, num of corrects: 706
# 17778.0


# Number of exact matches: 1550 out of 5000 , Percentage: 31.0
# Number of exact num of patent extracted: 3058 out of 5000
# Avg Total error: 3.0708, num of corrects:  3058
# Avg Total prophetic error: 1.7164, num of corrects: 2433
# Avg Total nonprophetic error: 3.9832, num of corrects: 1797
# 43852.0


Number of exact matches: 1550 out of 5000 , Percentage: 31.0
Number of exact num of patent extracted: 3058 out of 5000
Avg Total error: 3.0708, num of corrects:  3058
Avg Total prophetic error: 1.7164, num of corrects: 2433
Avg Total nonprophetic error: 3.9832, num of corrects: 1797
43852.0


,patentnumber,present,past,Unknown,prophetic,nonprophetic,allprophetic,someprophetic,Total_Extracted,Total_Freilich,prophetic_error,nonprophetic_error,Total_Mean_error,Sum_error
2445,8962623,3,66,420,0.0,483.0,0.0,0.0,69,483.0,3.0,417.0,414.0,834.0
277,8927710,29,227,353,0.0,564.0,0.0,0.0,256,564.0,29.0,337.0,308.0,674.0
3999,8987295,2,166,274,1.0,441.0,0.0,1.0,168,442.0,1.0,275.0,274.0,550.0
1843,8952157,2,1188,256,0.0,1446.0,0.0,0.0,1190,1446.0,2.0,258.0,256.0,516.0
1393,8946197,0,51,241,0.0,290.0,0.0,0.0,51,290.0,0.0,239.0,239.0,478.0
2868,8969341,4,200,231,0.0,435.0,0.0,0.0,204,435.0,4.0,235.0,231.0,470.0
2128,8957205,0,261,0,0.0,36.0,0.0,0.0,261,36.0,0.0,225.0,225.0,450.0
4859,9000054,5,509,195,0.0,709.0,0.0,0.0,514,709.0,5.0,200.0,195.0,400.0
3963,8987242,57,512,332,1.0,374.0,0.0,1.0,569,375.0,56.0,138.0,194.0,388.0
2427,8962598,8,19,193,0.0,214.0,0.0,0.0,27,214.0,8.0,195.0,187.0,390.0


In [33]:
1254/2000

0.627

In [ ]:

len(df_check[df_check.Total_Freilich == df_check.Total_Extracted]["Total_Mean_error"])

0

# ERRORS IN FREILICH DATASET:
patentnumber 8957069: in freilich 108.0	nonproph and 0 prophetic but there are many more examples than 108, our algorithm: 267 nonprophetic and 1 prophetic and 3 unknown
patent number 8946443:we extracted 271	freilich: 108, this patent has refrtence examples, are we extracting these or not? 
patent number 8952010: same issue do we take refrence examples? 
patent 8933099: there are far more than 33 examples in this patent but freilich says 33, our algorithm found 131
what to do with Preparation examples?? e.g 8962612 in freilich has 6 we got 70.

# improvement for our algorithm
1. remove duplicates somehow, e.g in patent number 08987242, there some examples are extracted twice just because the name e.g example aa137 appears multiple times.
2. better classification of the patents. e.g patent 8987295 has 442, we extract the same number as frelich but we clasify differently.
3. 

In [92]:
check_tense_nltk("""9-(4-(1-Aminoethyl)phenyl)-6-chloro-8-hydroxythieno[2,3-c]quinolin-4(5H)-o- ne Hydrochloride
(573) ##STR00933##
(574) Following General Procedure F, tert-butyl 1-(4-(6-chloro-8-methoxy-4-oxo-4,5-dihydrothieno[2,3-c]quinolin-9-yl)phen- yl)ethylcarbamate (50 mg, 0.10 mmol) was treated with BBr.sub.3 (1.0 M in CH.sub.2Cl.sub.2, 2 mL, 2 mmol) to afford the desired product (21 mg, 58%) as a light yellow solid (21 mg, 58%): .sup.1H NMR (500 MHz, CD.sub.4OD) .delta. 7.65 (dt, J=5.2, 3.4 Hz, 2H), 7.59 (d, J=5.4 Hz, 1H), 7.41 (dt, J=4.0, 2.6 Hz, 2H), 7.30 (s, 1H), 6.07 (d, J=: 5.4 Hz, 1H), 4.62 (q, J=6.8 Hz, 1H), 1.76 (d, J=6.9 Hz, 3H); ESI MS m/z 371 [C.sub.19H.sub.15ClN.sub.1O.sub.2S+H].sup.+; HPLC 97.8% (AUC), t.sub.R=9.72 min.""")

'past'

In [83]:
df_check[df_check.Unknown > 0]

,patentnumber,present,past,Unknown,prophetic,nonprophetic,allprophetic,someprophetic,Total_Extracted,Total_Freilich,prophetic_error,nonprophetic_error,Total_Mean_error
12,8946423,1,9,1,0.0,9.0,0.0,0.0,11,9.0,1.0,0.0,2.0
16,8946430,4,38,11,0.0,53.0,0.0,0.0,53,53.0,4.0,15.0,0.0
17,8946434,3,23,3,3.0,23.0,0.0,1.0,29,26.0,0.0,0.0,3.0
18,8946437,8,19,34,0.0,40.0,0.0,0.0,61,40.0,8.0,21.0,21.0


In [35]:
merged[merged.patentnumber == "8926083"][cols]

,patentnumber,xml,prophetic,nonprophetic,allprophetic,someprophetic
8,8926083,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...",0.0,49.0,0.0,0.0
